In [3]:
import ising_model as ising
import ssh_model as ssh
import long_range_ssh as lrssh
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
def find_fid(size, delta, **kwargs):
    h_list = np.linspace(-5, 5, sample_no)
    F = []
    sus = []
    degen = 0
    for i,h in enumerate(h_list):   # y-axis
        temp_F = []
        for j,k in enumerate(h_list):   # x-axis
            degen_1, s_1 = lrssh.states(L = size, U = k, V = h, **kwargs)
            degen_2, s_2 = lrssh.states(L = size, U = k+delta, V = h, **kwargs)
            fidelity = abs(np.dot(s_1,s_2))
            temp_F.append(fidelity)
            print("size: " + str(size) + "  sample no.: ", str(i), str(j))
        F.append(temp_F)
    return F, h_list


In [ ]:
x = np.arange(-8,8,2)
for dp in x:
    E, V = lrssh.states(L = 4, eta = 0.6, U = 0, V = dp, c = 0, d = 0)
    print(E)

In [ ]:
import long_range_ssh as mine
import kenny_lrssh as kenny

In [ ]:
L = 6
eta = -0.6
c = 0
d = 0
U = 0
V = -4

E_k, V_k = kenny.psi(c,d,2*L,U,V,eta)
E_m, V_m = mine.states(L,eta,c,d,U,V)

print(E_k - E_m)
print(V_k[0])
print(V_m[0])

print(abs(round(np.dot(V_k, V_m))) == 1)

print(E_m)

In [ ]:
size = np.linspace(0,100)
arr = []
for i in size:
    temp_k= []
    for j in size:
        k = i**2 + j
        temp_k.append(k)
    arr.append(temp_k)
    

In [ ]:
np.save('data/test_F.npy', arr)
np.save('data/test_x.npy', size)

In [ ]:
arr = np.random.rand(6,6)
arr

In [ ]:
arr[0,0]

In [13]:
from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import spinless_fermion_basis_1d # Hilbert space spin basis
import numpy as np # generic math functions
from scipy.sparse.linalg import eigsh
from scipy.sparse import csr_matrix
import time

os.environ['OMP_NUM_THREADS'] = '8' # set number of OpenMP threads to run in parallel
os.environ['MKL_NUM_THREADS'] = '8' # set number of MKL threads to run in parallel
os.environ['MKL_DEBUG_CPU_TYPE'] = '5' # utilize AVX2

def states(L,eta = 0, c = 1, d = 1, U = 0, V = 0):

    start = time.perf_counter()
    
    N = 2 * L
    basis=spinless_fermion_basis_1d(N, Nf = L)
    a = -(1+eta)
    b = -(1-eta)
    
    I1 = [[U,i,(i+1)%L] for i in range(0,L,2)]
    I2 = [[V,(i+1)%L,(i+2)%L] for i in range(0,L,2)]

    tA = [[+a,i,(i+1)%L] for i in range(0,L,2)]
    ta = [[-a,i,(i+1)%L] for i in range(0,L,2)]

    tB = [[+b,(i+1)%L,(i+2)%L] for i in range(0,L,2)]
    tb = [[-b,(i+1)%L,(i+2)%L] for i in range(0,L,2)]
    
    tC = [[+c,i,(i+3)%L] for i in range(0,L,2)]
    tc = [[-c,i,(i+3)%L] for i in range(0,L,2)]
    
    tD = [[+d,(i+1)%L,(i+4)%L] for i in range(0,L,2)]
    td = [[-d,(i+1)%L,(i+4)%L] for i in range(0,L,2)]

    static=[["nn",I1],["nn",I2],["+-",tA],["-+",ta],["+-",tB],["-+",tb],["+-",tC],["-+",tc],["+-",tD],["-+",td]]
    dynamic=[]

    no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)
    H=hamiltonian(static, dynamic, basis=basis, dtype=np.float64,**no_checks)
    DH = csr_matrix.todense(H)
    print(DH.size)
    con = time.perf_counter()
    ham = con - start

    E,V = eigsh(H.aslinearoperator(time=t0),k=4,which="SA")
    end = time.perf_counter()
    eigen = end - con
    return E, V[:,0], ham, eigen


In [14]:
from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import spin_basis_1d # Hilbert space spin basis
import numpy as np # generic math functions


# h:(0.25,2)

def isingstates(L,h):

    J = 1

    J_x = [[-J, i, (i+1)%L] for i in range(L)]
    h_z = [[-h, i] for i in range(L)]

    static = [["xx",J_x],["z",h_z]]
    dynamic = []

    basis = spin_basis_1d(L = L)

    no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)
    H = hamiltonian(static, dynamic, dtype=np.float64, basis = basis, **no_checks)

    E,V = H.eigsh(k = 1, which = 'SA')
    print(csr_matrix.todense(H).size)

    return E, V[:,0]


In [9]:
import time

In [16]:
t0 = 0
ham = 0
eigen = 0
start = time.perf_counter()
# E, V, __ham, __eigen= isingstates(L = 6, eta = 0.6, c = 0, d = 0, U = 0, V = -3)
E, V = isingstates(L = 10, h = 1.5)
# ham += __ham
# eigen += __eigen
end = time.perf_counter()
print(f"finished in {end - start:0.4f} seconds")
# print(ham)
# print(eigen)
print(E)

1048576
finished in 0.0222 seconds
[-16.72302491]
